In [86]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
from pathlib import Path
import os

def plot_something(df, x_col, y_col, title=None, x_col_name=None, y_col_name=None, filename=None):
    plt.figure(figsize=(12, 6))
    
    plt.plot(df[x_col], df[y_col], marker='o', label=f'{y_col} vs {x_col}')
    
    if x_col_name:
        plt.xlabel(x_col_name)
    else:
        plt.xlabel(x_col)
        
    if y_col_name:
        plt.ylabel(y_col_name)
    else:
        plt.ylabel(y_col)
        
    
    if title:
        plt.title(title)
    else:
        plt.title(f'{y_col} vs {x_col}')
    
    handles, labels = plt.gca().get_legend_handles_labels()
    if labels:
        plt.legend()
    
    plt.grid(True)
    
    if filename:
        plt.savefig(filename)
        plt.close()
    else:
        plt.show()

def plot_csv_exp1(df, dirpath=None, title_postfix=None):
   
    serial_time = df[df['Processes'] == 1]['ExecutionTime'].values[0]

    # Obliczenie przyspieszenia
    df['Speedup'] = serial_time / df['ExecutionTime']
    
    # Obliczenie efektywności
    df['Efficiency'] = df['Speedup'] / df['Processes']
    
    # Obliczenie serial fraction
    df['SerialFraction'] = (1 / df['Speedup'] - 1 / df['Processes']) / (1 - 1 / df['Processes'])

    # Wykres przyspieszenia
    plot_something(
        df, 
        "Processes", 
        "Speedup",
        title=f"Speedup vs Processes {title_postfix}" if title_postfix else None,
        filename=Path(dirpath)/"speedup.png" if dirpath else None)

    # Wykres efektywności
    plot_something(
        df, 
        "Processes", 
        "Efficiency", 
        title=f"Efficiency vs Processes {title_postfix}" if title_postfix else None,
        filename=Path(dirpath)/"efficiency.png" if dirpath else None)

    # Wykres serial fraction
    plot_something(
        df, 
        "Processes", 
        "SerialFraction", 
        y_col_name="Serial Fraction", 
        title=f"Serial Fraction vs Processes {title_postfix}" if title_postfix else "Serial Fraction vs Processes",
        filename=Path(dirpath)/"serial_fraction.png" if dirpath else None
        )


## Contact Order

### Logs

In [87]:
co_dir = Path("./contact_order")

csv_filename = co_dir / "logs.csv"

df_co_logs = pd.read_csv(csv_filename)

In [88]:
plot_something(
    df=df_co_logs, 
    x_col="end_idx", 
    y_col="checkpoint_duration",
    x_col_name="Numer of processed files",
    y_col_name="Checkpoint duration time [s]",
    filename=co_dir / "exec_time_vs_checkpoints.png"
    )

### Results

In [19]:
csv_filename = "./contact_order/contact_order_results.csv"

df_co_results = pd.read_csv(csv_filename)

In [28]:
error_percent = round(len(df_co_results[df_co_results["contact_order"].isna()])/len(df_co_results)*100, 2)

In [29]:
print(f"Error files: {error_percent}%")

Error files: 6.65%


## Download tests

In [96]:
def str2sec(time_str):
    h, m, s = map(int, time_str.split(":"))
    total_seconds = h * 3600 + m * 60 + s
    return total_seconds

source_dir = "/home/piotrek/Documents/personal/STUDIA_DS/semestr_2/LSC/projekt/download_tests"
dst_dir = "/home/piotrek/Documents/personal/STUDIA_DS/semestr_2/LSC/projekt/download_tests_results"


for dir_type in Path(source_dir).iterdir():
    if dir_type.is_dir():
        
        type_name = Path(dir_type).name
        results_type = pd.DataFrame([])
        
        error_results = []
        
        for dir_type_worker in dir_type.iterdir():
            if dir_type_worker.is_dir():
                match = re.search(r".*_(.*?)_.*", dir_type_worker.name)
                num_workers = match.group(1)
                df_type_worker = pd.read_csv(dir_type_worker / "download_progress.csv")
                df_type_worker["rel_time_sec"] = df_type_worker["rel_time"].apply(str2sec)
                total_exec_time = int(df_type_worker["rel_time_sec"].iloc[-1])
                df_type_worker["rel_time_sec"] = df_type_worker["rel_time_sec"].diff(periods=1)
                df_type_worker.dropna(inplace=True)
                
                df_type_worker_err = pd.read_csv(dir_type_worker / "download_error.txt")
                num_error = len(df_type_worker_err)
                error_results.append(num_error)
                
                save_dir_type_worker = Path(dst_dir) / os.path.relpath(dir_type_worker, source_dir)
                save_dir_type_worker.mkdir(parents=True, exist_ok=True)
                
                plot_something(
                    df=df_type_worker, 
                    x_col="num_file", 
                    y_col="rel_time_sec", 
                    title=f"{num_workers} workers - {type_name}",
                    filename=save_dir_type_worker/"relt_vs_numf.png"
                )
                
                results_type = pd.concat(
                    [
                        results_type,
                        pd.DataFrame.from_dict(
                            {
                                "Processes": [int(num_workers)],
                                "ExecutionTime": [total_exec_time]
                            }
                        )
                        ]
                    )
        
        total_files = int(df_type_worker["num_file"].iloc[-1]) + 1
        mean_type_error = round(sum(error_results)/(len(error_results)*total_files) * 100, 2)
        print(f"Error {type_name}: {mean_type_error}%")

            
        save_dir_type = Path(dst_dir) / os.path.relpath(dir_type, source_dir)
        save_dir_type.mkdir(parents=True, exist_ok=True)
        
        results_type = results_type.sort_values(by="Processes")
        
        plot_something(
                df=results_type, 
                x_col="Processes", 
                y_col="ExecutionTime", 
                title=f"Total download time vs number of processes - {type_name}",
                x_col_name="Processes [s]",
                filename=save_dir_type/"exect_vs_proc.png"
            )    
        plot_csv_exp1(results_type, dirpath=save_dir_type, title_postfix=f" - {type_name}")
            
                

Error storage_client: 6.1%
Error gsutil: 0.0%
Error gcloud_storage: 0.0%


Wnioski:

- limit plików na `$SCRATCH` - 1 mln
- ten zbiór wymaga zalogowanego użytkownika w google
- w związku z czym był problem np z wgetem (nie chciało mi się walczyć z tą autoryzacją)
- użycie każdej metody wymagało utworzenia projektu w Google Cloud oraz pozyskanie klucza API
- w związku z limitem plików zaimlementowaliśmy rozwiązanie batchowe - pobieranie, przetwarzanie, usuwanie plików w batchu
- `gsutil` oraz `gcloud storage` nie są dostępne w infrastrukturze plgrid, w związku z czym testy przeprowadzilismy lokalnie
- contact order liczymy niezależnie dla każdego wątku, w związku z czym zakładamy, że skaluje się liniowo, więc badaliśmy jedynie ograniczenia z tytułu równoległego pobierania plików
- istotne jest to, że `gsutil` oraz `gcloud storage` obciążały dekompresją system, stąd trwało to najprawdopodobniej dłużej. Szukałem jednak info jak robi to libka pythonowa i też wygląda na to, że stosuje kompresję i dekompresje przy przesyłaniu, ale nie jest to obciążające. Być może źle używałem tych dwóch libek i należało podać plik z manifestem, ale jedyne rozwiAzania jakie widzialem to np `gsutil -m cp -R gs://your-bucket .`, gdzie my nie chcemy calego bucketa xD. ewentualnie można było skleic jakiegos super długiego regexa, ale nie wiem, czy to dobry pomysł tworzyć tak długie zapytania xD
- jak coś, to wszystkie procesy były odpalane na procesorze 6C 12T, zmieniałem wszystkie parametry w `ThreadPool` z libki `concurrent`, mogłem wrzucic tak duzo procesów, ponieważ biblioteka robi kolejke dla tasków, a pobieranie trochę trwa i jest mało procesorożerne (chyba, że `gsutil` lub `google storage`, gdize faktycnzie ograniczenie ze strony procesora weszło)
- w przypadku libki pythonowej występowały błędy związane z pobieraniem (około ~6 plików), natomiast w przypadku plików pobieranych za pomocą `gsutil` oraz `google storage` - wszystkie pliki pobiarły się poprawnie